#### Question 1
Find the total number of flights for each month. The output should be in the following format:

Month | Number of Flights
--- | ---
1 | 123
2 | 456
... | ...

In [79]:
import pandas as pd
import datetime as dt
import itertools

flights_data = pd.read_csv ('drive/MyDrive/datasets/flightData.csv')

In [80]:
flights_data

,passengerId,flightId,from,to,date
0,48,0,cg,ir,2017-01-01
1,94,0,cg,ir,2017-01-01
2,82,0,cg,ir,2017-01-01
3,21,0,cg,ir,2017-01-01
4,51,0,cg,ir,2017-01-01
...,...,...,...,...,...
99995,14100,999,co,cg,2017-12-31
99996,9397,999,co,cg,2017-12-31
99997,7682,999,co,cg,2017-12-31
99998,12429,999,co,cg,2017-12-31


In [81]:
flights_data.isnull().sum()

passengerId    0
flightId       0
from           0
to             0
date           0
dtype: int64

In [82]:
flights_data.duplicated().sum()

0

In [83]:

flights_data['date'] = pd.to_datetime(flights_data['date'], errors='coerce')
monthly_count_df = flights_data['date'].groupby(flights_data.date.dt.month).agg('count').reset_index(name='count')

monthly_count_df.rename(columns={"date": "Month", "count": "Number of Flights"})

,Month,Number of Flights
0,1,9700
1,2,7300
2,3,8200
3,4,9200
4,5,9200
5,6,7100
6,7,8700
7,8,7600
8,9,8500
9,10,7600


#### Question 2
Find the names of the 100 most frequent flyers. The output should be in the following format:

PassengerId | Number of Flights | First Name | Last Name
--- | --- | --- | ---
123 | 100 | John | Doe
456 | 99 | Jane | Doe
... | ... | ... | ...

In [84]:
passengers_data = pd.read_csv ('drive/MyDrive/datasets/passengers.csv')

sorted_count_name_df = flights_data.groupby(['passengerId'])['flightId'].count().reset_index(name='Count').sort_values(['Count'], ascending=False)
sorted_count_name_df = sorted_count_name_df.head(100)
pd.merge(sorted_count_name_df, passengers_data, on='passengerId')

,passengerId,Count,firstName,lastName
0,2068,32,Yolande,Pete
1,1677,27,Katherina,Vasiliki
2,4827,27,Jaime,Renay
3,3173,26,Sunshine,Scott
4,8961,26,Ginny,Clara
...,...,...,...,...
95,3880,20,Kelly,Marlana
96,662,20,Aliza,Iesha
97,603,20,Raymon,Evelin
98,4774,20,Myles,Ivan


#### Question 3
Find the greatest number of countries a passenger has been in without being in the UK. For example, if the countries a passenger was in were: UK -> FR -> US -> CN -> UK -> DE -> UK, the correct answer would be 3 countries. The output should be in the following format:

PassengerId | Longest Run
--- | ---
123 | 5
456 | 10
... | ...

In [85]:
def split(sequence, sep):
  chunk = []
  for val in sequence:
    if val == sep:
      yield chunk
      chunk = []
    else:
      chunk.append(val)
  yield chunk

def find_max_list(list):
    list_len = [len(i) for i in list]
    return list_len[0]
from_to_data = flights_data[['passengerId','from','to']]
paths_data = from_to_data.groupby(['passengerId']).agg(lambda x: list(x))
longest_run = []
for index, row in paths_data.iterrows():
  zipped = list(itertools.chain.from_iterable(zip(row['from'], row['to'])))
  zipped = [k for k,_g in itertools.groupby(zipped)]
  seperated = list(split(zipped,'uk'))
  longest_run.append(find_max_list(seperated))
  
paths_data['Longest Run'] = longest_run
paths_data[['Longest Run']]

,Longest Run
passengerId,
1,6
2,2
3,5
4,9
5,2
...,...
15496,1
15497,1
15498,1


#### Question 4
Find the passengers who have been on more than 3 flights together. The output should be in the following format:

Passenger 1 Id | Passenger 2 Id | Number of Flights Together
--- | --- | --
123 | 456 | 4
789 | 101 | 5
... | ... | ...

In [87]:
flightIds = flights_data[['passengerId','flightId']]
flightId_list_data = flightIds.groupby(['passengerId']).agg(lambda x: list(x))

pd.DataFrame([
    [i1, i2, len(list(set(r1).intersection(r2)))]
    for (i1, r1), (i2, r2) in itertools.combinations(flightId_list_data.itertuples(), 2) if len(list(set(r1).intersection(r2))) > 3
], columns=['Passenger 1 Id', 'Passenger 2 Id', 'Number of Flights Together'])


,Passenger 1 Id,Passenger 2 Id,Number of Flights Together
0,1,37,4
1,1,38,4
2,1,76,4
3,1,120,4
4,1,1694,4
...,...,...,...
121348,15180,15188,4
121349,15180,15193,4
121350,15186,15188,5
121351,15186,15193,4


#### Question 5 (Optional)
Find the passengers who have been on more than N flights together within the range (from,to). The output should be in the following format:

Passenger 1 Id | Passenger 2 Id | Number of Flights Together | From | To
--- | --- | --- | --- | ---
123 | 456 | 5 | 2017-01-01 | 2017-03-03
789 | 123 | 6 | 2017-04-05 | 2017-05-06
... | ... | ... | ... | ...

The function should look something like:
```
def flownTogether(atLeastNTimes: int, from: date, to: date):
    '''
    Insert code here
    '''
```

In [54]:
def flownTogether(atLeastNTimes,from_date,to_date):
  mask = (flights_data['date'] > pd.to_datetime(from_date, errors='coerce')) & (flights_data['date'] <= pd.to_datetime(to_date, errors='coerce'))
  data = flights_data.loc[mask]
  # data = flights_data['date'].between(, , inclusive=False)
  flight_dates = data[['flightId','date']].drop_duplicates()

  flightIds = data[['passengerId','flightId']]
  flightId_list_data = flightIds.groupby(['passengerId']).agg(lambda x: list(x))
  return pd.DataFrame([
    [i1, i2, len(list(set(r1).intersection(r2))), flight_dates[flight_dates['flightId'].isin(list(set(r1).intersection(r2)))]['date'].min(),flight_dates[flight_dates['flightId'].isin(list(set(r1).intersection(r2)))]['date'].max()]
    for (i1, r1), (i2, r2) in itertools.combinations(flightId_list_data.itertuples(), 2) if len(list(set(r1).intersection(r2))) > atLeastNTimes
    ], columns=['Passenger 1 Id', 'Passenger 2 Id', 'Number of Flights Together','From','To'])

flownTogether(2,'2017-10-01','2017-12-10')

,Passenger 1 Id,Passenger 2 Id,Number of Flights Together,From,To
0,6,72,3,2017-10-08,2017-11-03
1,6,97,4,2017-10-08,2017-12-08
2,6,246,3,2017-10-08,2017-11-03
3,6,277,3,2017-10-08,2017-12-08
4,6,523,3,2017-10-08,2017-11-03
...,...,...,...,...,...
40491,14285,14291,3,2017-11-23,2017-12-02
40492,14285,14294,3,2017-11-23,2017-12-02
40493,14288,14291,3,2017-11-23,2017-12-02
40494,14288,14294,3,2017-11-23,2017-12-02


In [45]:
flight_dates = flights_data[['flightId','date']].drop_duplicates()

flightIds = flights_data[['passengerId','flightId']]
flightId_list_data = flightIds.groupby(['passengerId']).agg(lambda x: list(x))

def flownTogetherN(atLeastNTimes):
  return pd.DataFrame([
    [i1, i2, len(list(set(r1).intersection(r2))), flight_dates[flight_dates['flightId'].isin(list(set(r1).intersection(r2)))]['date'].min(),flight_dates[flight_dates['flightId'].isin(list(set(r1).intersection(r2)))]['date'].max()]
    for (i1, r1), (i2, r2) in itertools.combinations(flightId_list_data.itertuples(), 2) if len(list(set(r1).intersection(r2))) > atLeastNTimes
    ], columns=['Passenger 1 Id', 'Passenger 2 Id', 'Number of Flights Together','From','To'])

flownTogetherN(5)

,Passenger 1 Id,Passenger 2 Id,Number of Flights Together,From,To
0,6,72,8,2017-01-01,2017-11-03
1,17,2857,7,2017-06-27,2017-11-21
2,18,431,6,2017-03-14,2017-07-13
3,18,3177,7,2017-04-07,2017-11-17
4,18,13822,6,2017-10-15,2017-12-02
...,...,...,...,...,...
14571,14069,14087,7,2017-10-29,2017-12-31
14572,14069,14094,6,2017-10-29,2017-12-16
14573,14082,14087,7,2017-10-29,2017-12-31
14574,14082,14094,6,2017-10-29,2017-12-16
